In [109]:
import os
import gensim
from pathlib import Path
from gensim.models import Word2Vec
import numpy as np
# from gensim.summarization.textcleaner import get_sentences

from nltk.tokenize import sent_tokenize
from random import shuffle
import nltk
from gensim.utils import simple_preprocess
from nltk.tokenize import sent_tokenize
tagger = nltk.perceptron.PerceptronTagger()
import joblib
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from scipy.spatial.distance import cosine
import glob
from multiprocessing import Pool, freeze_support
import pandas as pd


# === Initialize NLP Tools ===

translator = str.maketrans('', '', punctuation) 
tagger = nltk.perceptron.PerceptronTagger()
stemmer = SnowballStemmer("english")


# === Set Working Directory ===

# Set your working directory (adjust this as needed)
wd = Path(r"C:\Users\sarah\OneDrive\Dokumente\Masterarbeit")
wd_models = wd / "models"
wd_results = wd / "results"


# === Define Folder Paths ===

# Make sure that you have these folders in your working directory
data_c = wd / "data"
data_temp = data_c / "temp"
data_freq = data_c / "freq"
data_dict = data_c / "dictionaries"
data_preprocessed = data_c / "preprocessed"
fig_dir = wd /"fig"

# Upload ressources
stopwords = joblib.load(data_c / "stopwords.pkl")
word_counts = joblib.load(data_freq / "word_counts_stemmed.pkl")
word_counts_weighted = joblib.load(data_freq / "word_counts_weighted.pkl")
affect_dic = joblib.load(data_dict / 'dictionary_affect.pkl')
cognition_dic = joblib.load(data_dict / 'dictionary_cognition.pkl')

In [68]:
os.chdir(data_temp)
cleaned_files = [
    str(data_temp / 'clean_speeches_indexed1.pkl'),
    str(data_temp / 'clean_speeches_indexed2.pkl'),
    str(data_temp / 'clean_speeches_indexed3.pkl'),
    str(data_temp / 'clean_speeches_indexed4.pkl')
]

In [70]:
def extract_sentences(dataname):
    data = joblib.load(dataname)
    data = [a[1] for a in data]  # keep only text, no id

    sentences = []
    for doc in data:
        sentences += sent_tokenize(doc)  # use nltk's sent_tokenize here

    sentences = [item for item in sentences if len(item.split()) > 1]
    sentences = [gensim.utils.simple_preprocess(item) for item in sentences]

    sentences = [[a for a in s if not a.isdigit()] for s in sentences]
    sentences = [[a for a in s if len(a) > 2] for s in sentences]

    sentences = [tagger.tag(s) for s in sentences]
    sentences = [[i[0] for i in s if i[1].startswith(('N', 'V', 'J'))] for s in sentences]

    sentences = [[stemmer.stem(i) for i in s] for s in sentences]
    sentences = [[a for a in s if a not in stopwords] for s in sentences]
    sentences = [[a for a in s if word_counts_stemmed[a] >= 10] for s in sentences]

    sentences = [s for s in sentences if len(s) > 1]  # eliminate empty
    shuffle(sentences)

    lab = dataname.replace('clean_speeches_', 'sentences_').replace('_.pkl', '.pkl')
    print(f'{dataname} processed')
    joblib.dump(sentences, lab)
    print(f'{lab} saved')

# Run for all your files
for fname in cleaned_files:
    extract_sentences(fname)

C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl processed
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\sentences_indexed1.pkl saved
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed2.pkl processed
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\sentences_indexed2.pkl saved
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed3.pkl processed
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\sentences_indexed3.pkl saved
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed4.pkl processed
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\sentences_indexed4.pkl saved


In [71]:
sentences_files = [
    'sentences_indexed1.pkl',
    'sentences_indexed2.pkl',
    'sentences_indexed3.pkl',
    'sentences_indexed4.pkl']

In [72]:
dataset = []

for dataname in sentences_files:  # <-- your list of sentence files
    data = joblib.load(dataname)
    dataset.extend(data)  # extend instead of append if you want all sentences in a single list

# === Model training ===
w2v = Word2Vec(
    sentences=dataset,    # iterator that loops over tokenized sentences
    vector_size=300,      # Word vector dimensionality (use `vector_size` in newer gensim)
    window=8,             # Context window size
    min_count=10,         # Minimum word count
    workers=8,            # Number of threads
    sample=1e-3,          # Downsample setting for frequent words
    epochs=10             # Number of iterations over the corpus
)

# Optimize memory usage (optional)
w2v.wv.fill_norms()  # only works in older gensim versions

# Save model
wd_models.mkdir(parents=True, exist_ok=True)  # create folder if it doesn't exist
w2v.save(str(wd_models / 'w2v-vectors_8_300.pkl'))

In [73]:
w2v = Word2Vec.load(str(wd_models / "w2v-vectors_8_300.pkl"))
word_vectors = w2v.wv # Embeddings

In [74]:
# === Calculate centroids ===

def findcentroid(text, model):
    vecs = [model.wv[w] * word_counts_weighted[w] for w in text if w in model.wv]
    vecs = [v for v in vecs if len(v) > 0]
    centroid = np.mean(vecs, axis=0)
    #centroid = centroid.reshape(1, -1)
    return centroid


affect_centroid = findcentroid(affect_dic, w2v)
cog_centroid = findcentroid(cognition_dic, w2v)

os.chdir(data_c)
joblib.dump(affect_centroid, 'centroids/affect_centroid.pkl')
joblib.dump(cog_centroid, 'centroids/cog_centroid.pkl')


['centroids/cog_centroid.pkl']

In [75]:
# Print the vectors
print("Affect centroid vector:\n", affect_centroid)
print("\nCognition centroid vector:\n", cog_centroid)

# Optional: shape and stats
print("\nShape of affect centroid:", affect_centroid.shape)
print("Shape of cognition centroid:", cog_centroid.shape)

print("\nAffect centroid summary: min, max, mean:", 
      np.min(affect_centroid), np.max(affect_centroid), np.mean(affect_centroid))
print("Cognition centroid summary: min, max, mean:", 
      np.min(cog_centroid), np.max(cog_centroid), np.mean(cog_centroid))

Affect centroid vector:
 [-0.00608798 -0.03492771  0.01405908  0.02630128 -0.04857672 -0.1278637
  0.1206017   0.24284929  0.02220609  0.06530752 -0.03228097 -0.03097843
  0.01043473 -0.14732435 -0.26938754  0.00691377  0.03254289  0.08645286
  0.16637743  0.05675362 -0.18768284 -0.01955027  0.01128447 -0.00183581
  0.27018696  0.03270628 -0.19852409 -0.09102426 -0.0511265  -0.11699844
  0.03494072 -0.14906853  0.02323899  0.02690839 -0.04481664  0.07235346
 -0.00102059 -0.26483372  0.08222571 -0.19196938 -0.02331756  0.02588117
  0.12817642 -0.00528961  0.1212317   0.0529688  -0.04989932 -0.05130346
 -0.12776153  0.10621514  0.07343408  0.03053066  0.04956501  0.04847677
 -0.0810184   0.04587177  0.06071044  0.05726838  0.05954647 -0.13661206
 -0.17127532  0.12201256 -0.00514129  0.01362167 -0.0237526  -0.00784076
  0.09162332 -0.02886288 -0.05266345 -0.00068738 -0.1333322   0.08877458
  0.09628373 -0.07945921 -0.03453472 -0.00555583 -0.1404627   0.02281174
 -0.02893426  0.11431761 -0

## Emotionality Score

In [1]:
# Set wd to data_preprocessed
os.chdir(data_preprocessed)

# === Load preprocessed speech data ===

preprocessed_final_files = [
    joblib.load(os.path.join(data_preprocessed, 'preprocessed_speeches_indexed1_final.pkl')),
    joblib.load(os.path.join(data_preprocessed, 'preprocessed_speeches_indexed2_final.pkl')),
    joblib.load(os.path.join(data_preprocessed, 'preprocessed_speeches_indexed3_final.pkl')),
    joblib.load(os.path.join(data_preprocessed, 'preprocessed_speeches_indexed4_final.pkl'))
]


NameError: name 'os' is not defined

In [78]:
###################################
# Define Functions              ###
###################################

# apparently is missing deleting intermediate files

def documentvecweight(lista):
    out = []
    lista = [i for i in lista if len(i[1]) > 0]
    for s in lista:
        vecs = [w2v.wv[w] * word_counts_weighted[w] for w in s[1] if w in w2v.wv]
        if len(vecs) == 0:
            a = np.nan
            c = np.nan
            score = np.nan
        else:
            v = np.mean(vecs, axis=0)
            a = cosine(v, affect_centroid)
            c = cosine(v, cog_centroid)
            score = (1 + 1 - a) / (1 + 1 - c)
        out.append([s[0], a, c, score])
    return out


def main_function(file_path, idx):
    dataset = joblib.load(file_path)
    data = documentvecweight(dataset)
    lab = os.path.join(data_c, f'temp_distances_main_{idx}.pkl')
    joblib.dump(data, lab)


###################################
#      Run main directly        ###
###################################

def main():
    files = [
        os.path.join(data_preprocessed, f'preprocessed_speeches_indexed{i+1}_final.pkl') #Changed!
        for i in range(4)
    ]
    for i, f in enumerate(files, start=1):
        main_function(f, i)

if __name__ == "__main__":
    main()


###################################
#      Recompose everything     ###
###################################

DATA_temp = [os.path.join(data_c, f'temp_distances_main_{i+1}.pkl') for i in range(4)]

tot = []
for dataname in DATA_temp:
    d = joblib.load(dataname)
    tot += d

tot_df = pd.DataFrame(tot, columns=['filename', 'affect_d', 'cognition_d', 'score'])
joblib.dump(tot_df, os.path.join(data_c, 'distances_10epochs.pkl'))


['C:\\Users\\sarah\\OneDrive\\Dokumente\\Masterarbeit\\data\\distances_10epochs.pkl']

In [79]:

# Print the first few rows
print(tot_df.head())

# Optionally, print the shape to see how many documents were processed
print("Shape:", tot_df.shape)

# Print a quick summary
print(tot_df.describe())

# Or print the full DataFrame (if small)
print(tot_df)

          filename  affect_d  cognition_d     score
0  SYR_04_1949.txt  0.392178     0.400979  1.005504
1  PRK_57_2002.txt  1.300450     0.974174  0.681938
2  LVA_76_2021.txt  1.200172     0.801189  0.667184
3  RWA_29_1974.txt  0.920350     0.533893  0.736406
4  BFA_26_1971.txt  1.026254     0.635050  0.713393
Shape: (1005, 4)
          affect_d  cognition_d        score
count  1005.000000  1005.000000  1005.000000
mean      1.130575     0.768413     0.701871
std       0.228257     0.156835     0.141048
min       0.392178     0.293237     0.405621
25%       0.999124     0.666527     0.598453
50%       1.164945     0.785873     0.687966
75%       1.303989     0.883953     0.788198
max       1.580940     1.231918     1.223759
             filename  affect_d  cognition_d     score
0     SYR_04_1949.txt  0.392178     0.400979  1.005504
1     PRK_57_2002.txt  1.300450     0.974174  0.681938
2     LVA_76_2021.txt  1.200172     0.801189  0.667184
3     RWA_29_1974.txt  0.920350     0.533893  

In [177]:
# Load your main corpus CSV 
un_corpus_merged = pd.read_csv(os.path.join(data_c, "un_corpus_merged.csv"), sep=';', encoding='utf-8') 
# Merge on filename 
un_corpus_merged = un_corpus_merged.merge(tot_df, on="filename", how="left")

# Optionally save back as pickle
joblib.dump(un_corpus_merged, os.path.join(data_c, "un_corpus_merged_with_scores.pkl"))

un_corpus_merged.to_csv(
    os.path.join(data_c, "un_corpus_merged_with_scores.csv"),
    sep=';', 
    index=False, 
    encoding='utf-8'
)

In [81]:
print(un_corpus_merged)

             filename                                             speech  \
0     SYR_04_1949.txt  Fayez EL-KHOURI Bey explained that although hi...   
1     PRK_57_2002.txt  ﻿I would like to\ncongratulate you, Sir, on yo...   
2     LVA_76_2021.txt  Let me begin by congratulating the Minister fo...   
3     RWA_29_1974.txt   Mr. President, your unanimous election to the...   
4     BFA_26_1971.txt  56.\t Mr. President, allow me to congratulate ...   
...               ...                                                ...   
1010  MNG_59_2004.txt  I\nwish to extend to you, Sir, my delegation’s...   
1011  CIV_61_2006.txt  First, Madam, I should like to congratulate yo...   
1012  PRY_50_1995.txt  At the outset, allow me to express my best\nwi...   
1013  FIN_72_2017.txt  Our thoughts today are with the people of Mexi...   
1014  SLB_63_2008.txt  At the outset, Sir, \nallow me to offer the wa...   

     country_code  year     country_name  speech_length_words  \
0             SYR  194